# mdx VMのデプロイとVCP既存サーバ (SSH) モード セットアップ

この Notebook では、mdx VM のデプロイと、その mdx VM をVCP既存サーバ(SSH) モードで利用できるようにするまでのセットアップを行います。

VCPで利用可能なクラウド環境にはAWSのようなパブリッククラウドや、学術機関のクラウド、オンプレミスシステム向けなど複数あります。  
mdx VM (仮想マシン) もこれらと同様にVCPで扱う計算資源に対応した背景や方式については以下の資料を参照してください。

* NIIオープンフォーラム2022 / クラウドトラック2 / 学認クラウドオンデマンド構築サービス・SINETStream  
  [VCP (Virtual Cloud Provider) の mdx サポートに向けた実装と活用例 (pdf)](https://www.nii.ac.jp/openforum/upload/20220601PM_Cloud2_02.pdf)

## 前提条件

1. VCコントローラが動作していること
    * このNotebookを実行しているJupyterNotebookサーバと同一ホスト上で稼働するVCPポータブル版のVCコントローラを使用
2. `vcp_config/vcp_config.yml` ファイルにおいてVCコントローラの設定が完了していること

## 準備

1. mdx REST API 認証トークン設定
2. mdx VM にSSHログインするためのキーペア作成
3. mdx の REST API エンドポイントに接続できることの確認

### mdx REST API 認証トークン設定

- mdx REST API 認証トークンは、[mdxユーザポータル](https://oprpl.mdx.jp/) の「トークン発行」により取得する。
  * 「トークン発行」は、ユーザポータル画面右上にあるユーザ名をクリックすると表示される。
  
![](./images/001-mdx-token.png)

In [ ]:
# mdx REST API 認証トークン設定
from getpass import getpass
mdx_token = getpass("mdx API token")

### mdx VM への SSH ログイン用キーペア作成

- mdx VM デプロイ時の設定項目に含まれる公開鍵を用意する。

In [ ]:
!test -f ~/.ssh/id_rsa || ssh-keygen -t rsa -f ~/.ssh/id_rsa -N ""
!ls -l ~/.ssh

### mdx REST API エンドポイント接続確認

- HTTPステータスコード 200 が返ることを確認する。

In [ ]:
# デフォルトのresolverがIPv6のアドレスを返すことにより以降のAPIで接続不可の場合があるため、以下のコードを実行しておく。
def use_ipv4_only():
    import socket
    old_getaddrinfo = socket.getaddrinfo
    def new_getaddrinfo(*args, **kwargs):
        responses = old_getaddrinfo(*args, **kwargs)
        return [response
                for response in responses
                if response[0] == socket.AF_INET]
    socket.getaddrinfo = new_getaddrinfo

use_ipv4_only()

In [ ]:
!curl https://oprpl.mdx.jp -w '%{http_code}\n' -o /dev/null -s

### （必要に応じて）Pythonパッケージ追加

- mdx VMのデプロイなど時間がかかる処理において、セル実行中の経過時間を表示するライブラリをインストールする。
  - <https://pypi.org/project/jupyter-autotime/>

In [ ]:
!pip install jupyter-autotime

## mdx VM 作成

1. VCP SDK mdx用プラグインモジュールの読み込み
2. mdx VM作成に必要なパラメータ設定
3. mdx VMデプロイ実行

### VCP SDK mdx用プラグインモジュールの読み込み

In [ ]:
from common import logsetting
from vcpsdk.plugins.mdx_ext import MdxResourceExt
mdx = MdxResourceExt(mdx_token)

### mdx VM作成に必要なパラメータ設定

- プロジェクトID
- ネットワークセグメントID
- sshログインのための公開鍵

自身が所属している（利用可能な）mdxのプロジェクト情報を確認する。

In [ ]:
import json
projects = mdx.get_assigned_projects()
print(json.dumps(projects[0]["projects"], indent=2))

以降のmdx操作対象とする「プロジェクト名」 (`name`) を設定する。

In [ ]:
project_name = ''
mdx.set_current_project_by_name(project_name)

操作対象として設定したプロジェクト情報を確認する。

In [ ]:
print(json.dumps(mdx.get_current_project(), indent=2))

プロジェクトで利用可能なットワークセグメントのリストを取得し、先頭のIDを設定する。

In [ ]:
segments = mdx.get_segments()
print(json.dumps(segments, indent=2))

segment_id = mdx.get_segments()[0]["uuid"]
print(segment_id)

sshログインのための公開鍵ファイルの内容を設定する

In [ ]:
import os
with open(os.path.expanduser('~/.ssh/id_rsa.pub')) as f:
    ssh_shared_key = f.read()
print(ssh_shared_key)

### mdx VMデプロイ

- ここでは、仮想マシンテンプレートとして「推奨版、東京大学制作、20220412-2043版」を使用する。
  * 利用可能な仮想マシンテンプレートの一覧は、 `get_vm_catalogs()` により確認できる。
- [ハードウェアのカスタマイズ項目](https://docs.mdx.jp/ja/main/create_vm.html#deploy-settings)は、以下の内容で設定する。
  * 仮想マシン名: `"handson-001"` **（プロジェクト内で重複しないこと）**
  * パックタイプ: `"cpu"`
  * パック数: `3`
  * 仮想ディスク(GB): `40`
  * ストレージネットワーク: `"portgroup"`

mdxVMにIPv4アドレスが付与されるまで5分程度かかるため、実行中のセルの経過時間を表示するライブラリ `jupyter-autotime` 機能を有効化しておく。

In [ ]:
%load_ext autotime

VMのデプロイを実行する。デプロイ完了後、VM情報を出力する。

In [ ]:
DEFAULT_CATALOG = "16a41081-a1cf-428e-90d0-a147b3aa6fc2"
DEFAULT_TEMPLATE_NAME = "UT-20220412-2043-ubuntu-2004-server"

vm_name = "handson_001"

mdx_spec = dict(
    catalog=DEFAULT_CATALOG,
    template_name=DEFAULT_TEMPLATE_NAME,
    pack_num=3,
    pack_type="cpu",
    disk_size=40,
    gpu="0",
    network_adapters=[
        dict(
            adapter_number=1,
            segment=segment_id
        )
    ],
    shared_key=ssh_shared_key,
    storage_network="portgroup",
)

info = mdx.deploy_vm(vm_name, mdx_spec)

print(json.dumps(info, indent=2))

In [ ]:
# jupyter-autotime を無効化
%unload_ext autotime

VMに付与されたプライベートIPv4アドレス (先頭のアドレス) を取得する。

In [ ]:
host_ip_address = info["service_networks"][0]["ipv4_address"][0]
host_ip_address

## VCP既存サーバ (SSH) モード セットアップ

1. mdx VMへの疎通確認
2. VM初期パスワード設定
3. 既存サーバ(SSH) モード セットアップスクリプトをmdx VMへのコピー
4. 既存サーバ(SSH) モード セットアップ実行
5. Docker のインストール確認

### mdx VMへの疎通確認

In [ ]:
!ping -c 3 {host_ip_address}

### VM初期パスワード設定

* 仮想マシンテンプレート「推奨版、東京大学制作、20220412-2043版」では、初回ログイン時にはOSのパスワード設定が求められる
* ログインユーザ名は `mdxuser`

VMの初期パスワードをSSH経由で設定するために必要なパラメータを設定する。

In [ ]:
ssh_user_name = "mdxuser"
ssh_private_key_path = os.path.expanduser("~/.ssh/id_rsa")
mdx_user_password = "handson2209_mdx_vm_initial_password"

expectスクリプトを用いてVMの初期パスワード設定を実行する。

In [ ]:
!./scripts/init_mdx_passwd.exp {ssh_user_name} {host_ip_address} {ssh_private_key_path} {mdx_user_password}

### 既存サーバ(SSH) モード セットアップスクリプトをmdx VMへのコピー

In [ ]:
# notebookからmdx VMへのsshコマンドオプション
ssh_opts = f" -i {ssh_private_key_path} -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null"

In [ ]:
!scp -i tmp/id_rsa -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null scripts/init_mdx_node.sh {ssh_user_name}@{host_ip_address}:

### 既存サーバ(SSH) モード セットアップ実行

- docker-ce インストール
- sshd_config 設定変更: Port 20022 

In [ ]:
!ssh {ssh_opts} {ssh_user_name}@{host_ip_address} ./init_mdx_node.sh

### Docker のインストール確認

In [ ]:
!ssh {ssh_opts} -p 20022 {ssh_user_name}@{host_ip_address} sudo docker ps

## VCコントローラ公開鍵設定

VCコントローラがmdx VMを既存サーバ(SSH) モードで制御するにはSSH接続を使用する。そのため、VCコントローラのSSH公開鍵をmdx VMに登録しておく必要がある。

1. VCP SDKの初期化
2. VCコントローラ公開鍵設定

### VCP SDKの初期化

VCP SDKを利用するにはVCコントローラのアクセストークンが必要となります。次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

OCSハンズオン用トークンはこちら→ [token.txt](/jupyter/files/token.txt)

In [ ]:
# VCP REST API アクセストークンの入力
from getpass import getpass
vcc_access_token = getpass()

In [ ]:
# VCP SDK 初期化
from vcpsdk.vcpsdk import VcpSDK
sdk = VcpSDK(vcc_access_token)

### VCコントローラ公開鍵取得設定

VCP SDK の「VCコントローラ公開鍵取得」APIを使って公開鍵を取得し、それをmdx VMにコピーする。

In [ ]:
# VCコントローラが既存サーバを操作するときに使うssh公開鍵を取得する
vcc_ssh_pubkey = sdk.get_publickey()

In [ ]:
# 公開鍵をmdx VMにコピーする
!echo "{vcc_ssh_pubkey}" | (ssh {ssh_opts} -p20022 {ssh_user_name}@{host_ip_address} "cat >> ~/.ssh/authorized_keys")

## VCP SDKによるVCノード起動確認

1. VCノード起動パラメータ設定
2. VCノード起動
3. Applicationコンテナの起動確認

### VCノード起動パラメータ設定

* クラウドプロバイダ名は `"onpremises"` を指定
* `ip_addresses`: Baseコンテナを起動するホストのIPアドレス
* `user_name`: Baseコンテナを起動するホスト上のsudo可能なアカウント名 (mdx VMテンプレート毎に既定）
* `set_ssh_pubkey`: BaseコンテナへのSSHログイン用公開鍵ファイルのパス

In [ ]:
my_ugroup_name = "mdx"
ugroup = sdk.create_ugroup(my_ugroup_name)

In [ ]:
print(ugroup)

In [ ]:
spec = sdk.get_spec("onpremises", "default")
spec.ip_addresses = [host_ip_address]
spec.user_name = ssh_user_name
spec.set_ssh_pubkey(os.path.expanduser("~/.ssh/id_rsa.pub"))

### VCノード起動

In [ ]:
unit_name = "mdx_server"
unit = ugroup.create_unit(unit_name, spec, wait_for=True, verbose=0)

In [ ]:
ugroup.df_nodes()

### Applicationコンテナの起動確認

`hello-world` コンテナを実行する。

In [ ]:
!ssh {ssh_opts} root@{host_ip_address} /usr/local/bin/docker run hello-world

## VCノード削除

ここでは VC ノードとしての動作確認を行った Unit Group の cleanup を実行するが、
VCP既存サーバ(SSH) モードをセットアップ済みの mdx VM は VC ノードとして再利用可能である。

In [ ]:
ugroup.cleanup()

In [ ]:
sdk.df_ugroups()